## 1. Bronze Layer: Load dataset, create database and table, insert the data into table

- Load the dataset into a pandas DataFrame.
- Create a SQLite database and a table for the dataset.
- Insert the raw data from the DataFrame into the SQLite table.

In [ ]:
import sqlite3
import pandas as pd

# 1. Load the dataset 

file_path = '/Users/lasya/Downloads/Popular_Baby_Names.csv'
df = pd.read_csv(file_path)

In [ ]:
df.columns

In [ ]:
# 2. Create SQLite Connection

conn = sqlite3.connect('Popular_Baby_Names.db')
cursor = conn.cursor()

In [ ]:
# Drop the existing table if it exists
cursor.execute("DROP TABLE IF EXISTS baby_names")

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS baby_names (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Year_of_Birth INTEGER,
    Gender TEXT,
    Ethnicity TEXT,
    Childs_First_Name TEXT,
    Count INTEGER,
    Rank INTEGER
)
''')

In [ ]:
# Prepare the data for insertion
baby_names_data = df[['Year of Birth', 'Gender', 'Ethnicity', "Child's First Name", 'Count', 'Rank']].to_records(index=False).tolist()

In [ ]:
# Insert the data into the baby_names table
cursor.executemany('INSERT INTO baby_names (Year_of_Birth, Gender, Ethnicity, Childs_First_Name, Count, Rank) VALUES (?, ?, ?, ?, ?, ?)', baby_names_data)

In [ ]:
# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('Popular_Baby_Names.db')
cursor = conn.cursor()

# Execute a query to select all data from the baby_names table
cursor.execute('''SELECT * FROM baby_names;''')

# Fetch and print all the results
rows = cursor.fetchall()

# Check if there are results and display them
if rows:
    for row in rows:
        print(row)
else:
    print("No data found.")

# Close the connection
conn.close()

## 2. Silver Layer: Data Cleaning & Validation 

- Read from the Bronze database.
- Clean the data by removing duplicates.
- Create a new Silver database with the cleaned data.

In [ ]:
conn = sqlite3.connect('Popular_Baby_Names.db')
df = pd.read_sql_query('SELECT * FROM baby_names', conn)

# Clean data: removing duplicates and null values
df_cleaned = df.drop_duplicates().dropna()
df_cleaned.columns = df_cleaned.columns.str.lower().str.replace(' ', '_')


# Create Silver database
conn_silver = sqlite3.connect('silver_popular_baby_names.db')
df_cleaned.to_sql('baby_names', conn_silver, if_exists='replace', index=False)
conn.close()
conn_silver.close()

In [ ]:
conn_silver = sqlite3.connect('silver_popular_baby_names.db')
cursor = conn_silver.cursor()
cursor.execute('''SELECT * FROM baby_names;''')

# Fetch and print all the results
rows = cursor.fetchall()

# Check if there are results and display them
if rows:
    for row in rows:
        print(row)
else:
    print("No data found.")
conn_silver.close()

## 3. Gold Layer: The final stage with enriched, business-ready data.

- Read from the Silver database.
- Create summary statistics (e.g., total count of babies by name and year).
- Store this summary in the Gold database.

In [ ]:
conn_silver = sqlite3.connect('silver_popular_baby_names.db')
df = pd.read_sql_query('SELECT * from baby_names', conn_silver)

# Create a summary table: number of babies by name and year
df_summary = df.groupby(['year_of_birth', 'childs_first_name']).agg({'count': 'sum'}).reset_index()
df_summary.columns = ['year_of_birth', 'childs_first_name', 'total_count']

# Create Gold database
conn_gold = sqlite3.connect('gold_popular_baby_names.db')
df_summary.to_sql('baby_names_summary', conn_gold, if_exists='replace', index=False)
conn_silver.close()
conn_gold.close()

In [ ]:
conn_gold = sqlite3.connect('gold_popular_baby_names.db') #try using func
cursor = conn_gold.cursor()
cursor.execute('''SELECT * FROM baby_names_summary;''')

# Fetch and print all the results
rows = cursor.fetchall()

# Check if there are results and display them
if rows:
    for row in rows:
        print(row)
else:
    print("No data found.")
conn_gold.close()